In [2]:
# 바른 자세(0)
### 원본 + 어깨 한 쪽만 찍히는 경우 (조건문)
### 어깨 양쪽 모두 안찍히면, 직전 값 누적


import cv2
import time
import dlib
import numpy as np
import pandas as pd
import time
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)


end_time = time.time() + (60*1)
   
def Detection():        
    # Pose Detection : shoulder 2번, 5번
    MODE = "MPI"

    if MODE is "MPI" :
        protoFile = "pose_deploy_linevec_faster_4_stages.prototxt"
        weightsFile = "pose_iter_160000.caffemodel"
        nPoints = 7

    threshold = 0.1
    net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)

    # Face Detection : 1번 ~ 68번
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

    # WebCam(0) 으로 이미지 추출
    cap = cv2.VideoCapture(0)
    time.sleep(1) 

    # WebCam 실행 X
    if not cap.isOpened():
      exit()

    shoulder_result = []
    face_result = []
    face_x_result = []
    # WebCam 실행 O
    while True:
        hasFrame, image = cap.read()
        #cv2.imshow('image', image)
        frameCopy = np.copy(image)
        faces = detector(image)

        if not hasFrame:
            break

        frameHeight = image.shape[0]

        inpBlob = cv2.dnn.blobFromImage(image, 1.0 / 255, (368, 368),(0, 0, 0), swapRB=False, crop=False)
        net.setInput(inpBlob)
        output = net.forward()

        H = output.shape[2]

        # Empty list to store the detected keypoints
        shoulder_points = []   
        face_points = []

        # Pose Detection : shoulder
        for i in range(nPoints):
            # confidence map of corresponding body's part.
            probMap = output[0, i, :, :]

            # Find global maxima of the probMap.
            minVal, prob, minLoc, point = cv2.minMaxLoc(probMap)

            # Scale the point to fit on the original image
            y = (frameHeight * point[1]) / H
            
            
            if prob > threshold:
                shoulder_points.append((float(y)))
            else:
                shoulder_points.append(None)

        # pose detection에서 어깨(2번, 5번) y좌표의 평균 -> mean_shoulder    
        if shoulder_points[2] != None:
            if  shoulder_points[5] != None:
                mean_shoulder = (shoulder_points[2] + shoulder_points[5])/2
            elif shoulder_points[5] == None:
                means_shoulder = shoulder_points[2]
            else:
                pass
        elif shoulder_points[2] == None:
            if  shoulder_points[5] != None:
                means_shoulder = shoulder_points[5]

            # 어깨 양쪽이 모두 추출 안될 때 : 웹캠 재 실행
            elif shoulder_points[5] == None:
                print("양쪽 어깨가 화면에 보이도록 자세를 조정해주세요!")
            else:
                pass

        # shoulder_result에 mean_shoulder값 추가
        shoulder_result.append(mean_shoulder)
        
        # face detection
        for face in faces:
            landmarks = predictor(image, face)
            for n in range(0, 36):
                y = landmarks.part(n).y
                face_points.append(y)
               
        face_result.append(face_points)

        # 종료
        if cv2.waitKey(1) == ord('q'):
            break
            
        if time.time() > end_time:
                print("실험이 종료 되었습니다.")
                cap.release()
                cv2.destroyAllWindows()
                
                shoulder_result = pd.DataFrame(shoulder_result)
                face_result = pd.DataFrame(face_result)
                
                """#얼굴 광대 길이 df
                face_x_result = pd.DataFrame(face_x_result)
                face_x_result.columns = ["x_length"]"""
               
            
                #pd를 array로 
                face_result = face_result.to_numpy()
                shoulder_result = shoulder_result.to_numpy()
                
                """print(face_result)
                print(face_result.shape)
                print(shoulder_result)
                print(shoulder_result.shape)"""
                #array-array(얼굴과 어꺠사이의 거리)
                subtract = face_result-shoulder_result
                df = pd.DataFrame(subtract)
                """print(subtract)
                print(subtract.shape)
                print(face_x_result)
                print(face_x_result.shape)"""
                
                #df:최종
                #subtract(array)->df
                df["class"] = 1
                df.to_csv("./A_data/low/1/1_train_0523_psy.csv", index = False)
                break
           
Detection()


실험이 종료 되었습니다.


In [22]:
#low_1은 low_0복사한것이지만 x_length는 뽑아낼 필요가 없어서 과정없음
#변수명
"""shoulder_result : 어깨선 평균값 담음
face_result : 얼굴좌표 담음
face_x_result : 얼굴광대길이 담음(dataframe)
x_length : 얼굴광대길이
subtract : 얼굴좌표 - 어깨선 좌표
최종 : df"""

'shoulder_result : 어깨선 평균값 담음\nface_result : 얼굴좌표 담음\nface_x_result : 얼굴광대길이 담음(dataframe)\nx_length : 얼굴광대길이\nsubtract : 얼굴좌표 - 어깨선 좌표\n최종 : df'